In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from lib import model_device_io as io
from lib import data_loader as dl
from lib import models as md

In [11]:
trainset = dl.HotelReservationData(root='data/train')
testset = dl.HotelReservationData(root='data/test')
train_loader = DataLoader(trainset, batch_size=4096, shuffle=True, num_workers=0)
test_loader = DataLoader(testset, batch_size=4096, shuffle=False, num_workers=0)
device = io.getCudaDevice(cudaNum = 1, torchSeed = 123)
model = md.Classifier(input_size = 258).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
io.loadModel('models/rev_best.pth', model, optimizer)

Device used: cuda:1
model loaded from models/rev_best.pth


In [12]:
def train(train_epoch = 100, model_fn = "rev_best.pth", log_per_epoch = 4):
    best_acc = test()
    for epoch in range(train_epoch):
        model.train()
        for batch_idx, (data, _, rev) in enumerate(train_loader):
            data, rev = data.to(device), rev.to(device)
            output = model(data)
            loss = F.cross_entropy(output, rev)
            loss.backward()
            optimizer.step()
            if (batch_idx+1) % max(1, int(len(train_loader)/log_per_epoch)) == 0:
                print('Training Epoch: {:.2f}% ({}/{})\tBatch: {:.2f}% ({}/{})\tLoss: {:.6f}'.format(
                      (epoch+1)*100./train_epoch, epoch+1, train_epoch, (batch_idx+1)*100./len(train_loader),
                      batch_idx+1, len(train_loader), loss.item()))
        acc = test()
        if(best_acc < acc):
            best_acc = acc
            fn = os.path.join('models', model_fn)
            io.saveModel(fn, model, optimizer)
            
            

def test():
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for data, _, rev in test_loader:
            data, rev = data.to(device), rev.to(device)
            output = model(data)
            loss += F.cross_entropy(output, rev).item()*len(data) # sum up batch loss
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(rev.view_as(pred)).sum().item()
            
    loss /= len(test_loader.dataset)
    acc = 1.*correct/len(test_loader.dataset)
    print('***Validation Results***\tLoss: {:.4f},\tAccuracy: {:.2f}% ({}/{})\n'.format(
          loss, acc*100., correct, len(test_loader.dataset)))
    return acc

In [13]:
train(train_epoch = 10)

***Validation Results***	Loss: 11.5085,	Accuracy: 64.30% (6430/10000)

Training Epoch: 10.00% (1/10)	Batch: 25.00% (5/20)	Loss: 21.555044
Training Epoch: 10.00% (1/10)	Batch: 50.00% (10/20)	Loss: 23.809071
Training Epoch: 10.00% (1/10)	Batch: 75.00% (15/20)	Loss: 22.959814
Training Epoch: 10.00% (1/10)	Batch: 100.00% (20/20)	Loss: 20.446157
***Validation Results***	Loss: 12.1257,	Accuracy: 64.30% (6430/10000)

Training Epoch: 20.00% (2/10)	Batch: 25.00% (5/20)	Loss: 22.276583
Training Epoch: 20.00% (2/10)	Batch: 50.00% (10/20)	Loss: 19.837347
Training Epoch: 20.00% (2/10)	Batch: 75.00% (15/20)	Loss: 24.895195
Training Epoch: 20.00% (2/10)	Batch: 100.00% (20/20)	Loss: 22.160988
***Validation Results***	Loss: 12.2111,	Accuracy: 64.30% (6430/10000)

Training Epoch: 30.00% (3/10)	Batch: 25.00% (5/20)	Loss: 23.479626
Training Epoch: 30.00% (3/10)	Batch: 50.00% (10/20)	Loss: 23.246603
Training Epoch: 30.00% (3/10)	Batch: 75.00% (15/20)	Loss: 22.568569
Training Epoch: 30.00% (3/10)	Batch: 100